In [10]:
## Neo4j Init + query function definition

from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Use os.environ to access environment variables
NEO4J_URI = os.environ.get('NEO4J_URI')
NEO4J_USERNAME = os.environ.get('NEO4J_USERNAME') # default neo4j
NEO4J_PASSWORD = os.environ.get('NEO4J_PASSWORD') # default neo4j for local instance
AURA_INSTANCEID = os.environ.get('AURA_INSTANCEID')
AURA_INSTANCENAME = os.environ.get('AURA_INSTANCENAME')

# Now you can use `api_key` in your code
print(NEO4J_URI)

# print password fragment for a visual check
print(NEO4J_PASSWORD[:4] + '...'+ NEO4J_PASSWORD[-4:])  # Just for demonstration, remove or replace this in actual use

# Define a simple query function
from typing import List, Optional, Dict, Any
def run_query(driver: Driver, query:str, parameters:Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
    """
    Executes a Cypher query against the Neo4j database using the provided driver.
    
    Args:
        query (str): The Cypher query to be executed.
        parameters (Optional[Dict[str, Any]]): A dictionary of parameters for the Cypher query.
        driver (Driver): An instance of a Neo4j driver to manage connections to the database.
        
    Returns:
        List[Dict[str, Any]]: A list of dictionaries, where each dictionary represents the fields
        of a single record returned by the query.
        
    Raises:
        Exception: If an error occurs during query execution or session management.
    """
    try:
        with driver.session() as session:
            result = session.run(query, parameters)
            return [dict(record) for record in result]
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

neo4j+s://8bb749c2.databases.neo4j.io
QqPm...g-5Q


In [14]:
# 1. first we need to prompt OpenAI so that its response structure is either JSON or JSON is included in the response.
# 
# 2. Then we need to pull our JSON structure from the message (unless the message contains only our JSON and nothing else). Either way it won't harm to have extra check
import re
import json

def extract_first_json(text):
    # Regex pattern to find the first instance of a JSON object or array
    json_pattern = r'(\{.*?\}|\[.*?\])'
    
    # Using non-greedy matching to capture the smallest possible valid JSON string
    match = re.search(json_pattern, text, re.DOTALL)
    if match:
        json_str = match.group(0)
        try:
            # Convert the JSON string to a Python dictionary or list
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError:
            print("Found string is not a valid JSON.")
    else:
        print("No JSON object or array found in the text.")

# Example usage
openai_api_response_text = '''
    Here is a JSON object: [{"name":"Chelsea", "description": "Football Club"}, {"name":"Player", "description":"Football Player"}, {"name":"Coach", "description":"Football Team Coach"}, {"name":"Stadium", "description":"Football Stadium"}]
    Let me know if you need any more help
    Yours truly,
    OpenAI
    P.S. Big Brother is watching you
'''
openai_reposponse_json = extract_first_json(openai_api_response_text)

print(openai_reposponse_json)



# 3. Connect to Neo4j
from neo4j import GraphDatabase
from neo4j import Driver

# Replace these with your Aura or Neo4j connection details or use the values loaded from environment variables.
uri = NEO4J_URI
username = NEO4J_USERNAME
password = NEO4J_PASSWORD

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# 4. Empty database for a clean test
from neo4j import GraphDatabase

# Replace these with your Aura connection details
uri = NEO4J_URI
username = NEO4J_USERNAME
password = NEO4J_PASSWORD

class DatabaseCleaner:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def reset_database(self):
        with self.driver.session() as session:
            session.write_transaction(self._delete_all)

    @staticmethod
    def _delete_all(tx):
        tx.run("MATCH (n) DETACH DELETE n")

if __name__ == "__main__":
    db_cleaner = DatabaseCleaner(uri, username, password)
    db_cleaner.reset_database()
    db_cleaner.close()
    print("Database has been reset to a blank state.")


# 5. Load JSON you got from the API response as a bunch of entities in Neo4j

for entity_and_description_pair in openai_reposponse_json:
    # create the first entity
    entity_name = entity_and_description_pair["name"]
    entity_description = entity_and_description_pair["description"]
    neo4j_cyper_query = f'CREATE ({entity_name}:Entity {{name: "{entity_name}", description: "{entity_description}"}})'
    print(neo4j_cyper_query)
    results = run_query(driver, neo4j_cyper_query)
    print(results)


# 6. Check if it worked


# check the database is empty
query = "MATCH (n) RETURN n LIMIT 5"
results = run_query(driver, query)
print('you should see the output is empty, e.g. "[ ]" \nOtherwise see the code for database cleanup at the bottom of this notebook')
print(results)


[{'name': 'Chelsea', 'description': 'Football Club'}, {'name': 'Player', 'description': 'Football Player'}, {'name': 'Coach', 'description': 'Football Team Coach'}, {'name': 'Stadium', 'description': 'Football Stadium'}]


C:\Users\sergey.lyubarskiy\AppData\Local\Temp\ipykernel_38892\3730250134.py:67: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._delete_all)


Database has been reset to a blank state.
CREATE (Chelsea:Entity {name: "Chelsea", description: "Football Club"})
[]
CREATE (Player:Entity {name: "Player", description: "Football Player"})
[]
CREATE (Coach:Entity {name: "Coach", description: "Football Team Coach"})
[]
CREATE (Stadium:Entity {name: "Stadium", description: "Football Stadium"})
[]
you should see the output is empty, e.g. "[ ]" 
Otherwise see the code for database cleanup at the bottom of this notebook
[{'n': <Node element_id='4:75348602-e7db-421d-bac5-d06e7457432b:2' labels=frozenset({'Entity'}) properties={'name': 'Chelsea', 'description': 'Football Club'}>}, {'n': <Node element_id='4:75348602-e7db-421d-bac5-d06e7457432b:3' labels=frozenset({'Entity'}) properties={'name': 'Player', 'description': 'Football Player'}>}, {'n': <Node element_id='4:75348602-e7db-421d-bac5-d06e7457432b:4' labels=frozenset({'Entity'}) properties={'name': 'Coach', 'description': 'Football Team Coach'}>}, {'n': <Node element_id='4:75348602-e7db-4

In [15]:
# Some basic similarity search to be refined, but proves the index works
# More information here: https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/vector-indexes/#:~:text=You%20can%20create%20vector%20indexes%20using%20the%20CREATE,must%20be%20unique%20among%20both%20indexes%20and%20constraints.
search
query = '''MATCH (n)
WHERE toLower(n.description) CONTAINS 'club'
RETURN n.name'''
results = run_query(driver, query)
print(results)

[{'n.name': 'Chelsea'}]


In [22]:
additional_information_to_append_to_user_prompt_before_sending_to_completion_api = results[0]['n.name']

In [25]:
user_prompt = 'What and where and how much?'
user_prompt_enriched = f'''
# TASK
Answer the question of the user using the best of your knowledge as of your last training data, combined with the additional information included in the prompt under the user questions below.

# CONTEXT
### The user has asked the following question
{user_prompt}

### Corporate database contains the following additional information that may be relevant to the query
{additional_information_to_append_to_user_prompt_before_sending_to_completion_api}
'''

In [26]:
print(user_prompt_enriched)


# TASK
Answer the question of the user using the best of your knowledge as of your last training data, combined with the additional information included in the prompt under the user questions below.

# CONTEXT
### The user has asked the following question
What and where and how much?

### Corporate database contains the following additional information that may be relevant to the query
Chelsea

